In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
import os
import glob
import codecs
import csv
import prepare

In [2]:
#display all columns 
pd.set_option("display.max_columns", None)

In [3]:
#pulling in main csv
df = pd.read_csv('ATPTotal.csv')
df.head(1)

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,0,2019-M020,Brisbane,Hard,32,A,20181231,300,105453,2.0,NaN,Kei Nishikori,R,178.0,JPN,29.004791,106421,4.0,NaN,Daniil Medvedev,R,198.0,RUS,22.885695,6-4 3-6 6-2,3,F,124.0,3.0,3.0,77.0,44.0,31.0,17.0,13.0,3.0,6.0,8.0,6.0,100.0,54.0,34.0,20.0,14.0,10.0,15.0,9.0,3590.0,16.0,1977.0


In [4]:
df = df.set_index('tourney_date')
# convert index to datetime object and sort in ascending order
df.index = pd.to_datetime(df.index, format = '%Y%m%d')
df = df.sort_index(ascending = True)
df = df['1999-01-01':'2020-01-01']
# drop all walkovers (no useful stats) and best of 1 matches (extremely rare format)
df = df.drop(df[df.score == 'W/O'].index)
df = df.drop(df[df.best_of == 1].index)

In [5]:
df.head(1)

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
tourney_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-11,108,1999-338,Sydney,Hard,32,A,16,103720,NaN,NaN,Lleyton Hewitt,R,180.0,AUS,17.878166,102158,2.0,NaN,Patrick Rafter,R,185.0,AUS,26.036961,7-6(1) 6-1,3,R32,84.0,2.0,0.0,73.0,51.0,32.0,14.0,10.0,4.0,6.0,0.0,5.0,59.0,36.0,24.0,8.0,9.0,3.0,7.0,104.0,456.0,4.0,3315.0


In [6]:
#pulling 
#Create dataframe for Player Data and label columns
PlayerColumns = ['PlayerID', 'Player_Name', 'Age', 'Height', 'MaxRank', 'Hand', 'Country']

PlayerData = pd.DataFrame(columns = PlayerColumns)
PlayerData.set_index('PlayerID')


#Add data to the PlayerData dataframe
for index, row in df.iterrows():
    #player was the winner
    p_id = int(row.winner_id)
    
    #Check value
    if(p_id not in PlayerData.index):
        PlayerData.loc[p_id] = [p_id, row.winner_name, row.winner_age, row.winner_ht, row.winner_rank, row.winner_hand, row.winner_ioc]

    #Update rank with best rank value
    if(row.winner_rank < PlayerData.loc[p_id].MaxRank and not np.isnan(PlayerData.loc[p_id].MaxRank)):
        PlayerData.loc[p_id] = [p_id, row.winner_name, row.winner_age, row.winner_ht, row.winner_rank, row.winner_hand, row.winner_ioc]

    #Player was loser
    p2_id = int(row.loser_id)
    #Check for loser
    if(p2_id not in PlayerData.index):
        PlayerData.loc[p2_id] = [p2_id, row.loser_name, row.loser_age, row.loser_ht, row.loser_rank, row.loser_hand, row.loser_ioc]

    #Update rank with best rank value
    if(row.loser_rank < PlayerData.loc[p2_id].MaxRank and not np.isnan(PlayerData.loc[p2_id].MaxRank)):
        PlayerData.loc[p2_id] = [p2_id, row.loser_name, row.loser_age, row.loser_ht, row.loser_rank, row.loser_hand, row.loser_ioc]


In [7]:
#looking at main player details
PlayerData

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country
103720,103720,Lleyton Hewitt,20.752909,180.0,1.0,R,AUS
102158,102158,Patrick Rafter,26.477755,185.0,2.0,R,AUS
103017,103017,Nicolas Kiefer,22.535250,183.0,4.0,R,GER
210013,210013,Martin Damm Sr,28.227242,188.0,67.0,R,CZE
103264,103264,Mariano Puerta,26.904860,180.0,9.0,L,ARG
...,...,...,...,...,...,...,...
144984,144984,Artem Dubrivnyy,20.700890,NaN,424.0,R,RUS
200553,200553,Alen Avidzba,19.635866,NaN,498.0,R,RUS
208230,208230,Alibek Kachmazov,17.158111,NaN,1491.0,R,RUS
127339,127339,Borna Gojo,21.722108,NaN,280.0,R,CRO


In [8]:
# Win Count DF
win_count = df.groupby('winner_id')[['winner_name']].agg('count').sort_values(by='winner_id').astype(int).reset_index()
win_count = win_count.rename(columns={'winner_name': 'win_count', 'winner_id' : 'PlayerID'})
win_count

,PlayerID,win_count
0,100644,149
1,100754,3
2,100943,1
3,101086,8
4,101150,39
...,...,...
1444,208028,1
1445,208029,2
1446,208055,1
1447,209188,1


In [9]:
# Loss Count DF
loss_count = df.groupby('loser_id')[['loser_name']].agg('count').sort_values(by='loser_id').astype(int).reset_index()
loss_count = loss_count.rename(columns={'loser_name': 'lose_count', 'loser_id' : 'PlayerID'})
loss_count

,PlayerID,lose_count
0,100644,71
1,100754,4
2,100943,2
3,100997,2
4,101086,19
...,...,...
2133,208597,1
2134,208937,1
2135,209226,1
2136,209238,1


In [10]:
### merge win_loss counts

from functools import reduce

data_frames = [win_count, loss_count]

win_loss = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)

win_loss = win_loss.dropna()
win_loss['match_count'] = win_loss['win_count'] + win_loss['lose_count']
win_loss['win%'] = round((win_loss['win_count'] / win_loss['match_count']),2)

win_loss

,PlayerID,win_count,lose_count,match_count,win%
0,100644,149,71.0,220.0,0.68
1,100754,3,4.0,7.0,0.43
2,100943,1,2.0,3.0,0.33
3,101086,8,19.0,27.0,0.30
4,101150,39,55.0,94.0,0.41
...,...,...,...,...,...
1440,206173,9,5.0,14.0,0.64
1441,207139,2,2.0,4.0,0.50
1445,208029,2,1.0,3.0,0.67
1446,208055,1,1.0,2.0,0.50


In [11]:
#First Serve win percentage added new column
df['w_first_serve'] = df['w_1stIn'] / df['w_svpt']
df['l_first_serve'] = df['l_1stIn'] / df['l_svpt']

#First Serve Win
df['w_first_serve_win'] = df['w_1stWon'] / df['w_1stIn']
df['l_first_serve_win'] = df['l_1stWon'] / df['l_1stIn']


In [12]:


loser_first_serve_percent = df.groupby('loser_id')[['l_first_serve']].mean().reset_index()
loser_first_serve_percent = loser_first_serve_percent.rename(columns={'loser_first_serve_percent': 'l_first_serve', 'loser_id' : 'PlayerID'})

winner_first_serve_percent = df.groupby('winner_id')[['w_first_serve']].mean().reset_index()
winner_first_serve_percent = winner_first_serve_percent.rename(columns={'winner_first_serve_percent': 'w_first_serve', 'winner_id' : 'PlayerID'})



loser_first_serve_win_percent = df.groupby('loser_id')[['l_first_serve_win']].mean().reset_index()
loser_first_serve_win_percent = loser_first_serve_win_percent.rename(columns={'loser_first_serve_win_percent': 'l_first_serve_win', 'loser_id' : 'PlayerID'})

winner_first_serve_win_percent = df.groupby('winner_id')[['w_first_serve_win']].mean().reset_index()
winner_first_serve_win_percent = winner_first_serve_win_percent.rename(columns={'winner_first_serve_win_percent': 'w_first_serve_win', 'winner_id' : 'PlayerID'})

### merge serve stats

from functools import reduce

data_frames = [loser_first_serve_percent, winner_first_serve_percent, loser_first_serve_win_percent, winner_first_serve_win_percent]

serve_pts = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
serve_pts

,PlayerID,l_first_serve,w_first_serve,l_first_serve_win,w_first_serve_win
0,100644,0.617291,0.645815,0.654301,0.778817
1,100754,NaN,NaN,NaN,NaN
2,100943,NaN,NaN,NaN,NaN
3,100997,NaN,NaN,NaN,NaN
4,101086,0.644105,0.676382,0.591278,0.702205
...,...,...,...,...,...
2133,208597,NaN,NaN,NaN,NaN
2134,208937,NaN,NaN,NaN,NaN
2135,209226,NaN,NaN,NaN,NaN
2136,209238,NaN,NaN,NaN,NaN


In [13]:
# from functools import reduce

# data_frames = [PlayerData, win_loss, serve_pts]

# PD_merge = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
#                                             how='left'), data_frames)
# PD_merge

In [14]:
### Breakpoints
#df.head(1)
df['w_break_points_won'] = df['l_bpFaced']- df['l_bpSaved']
df['l_break_points_won'] = df['w_bpFaced']- df['w_bpSaved']

loser_breakpoints_won = df.groupby('loser_id')[['l_break_points_won']].agg('count').reset_index()
loser_breakpoints_won = loser_breakpoints_won.rename(columns={'loser_id' : 'PlayerID'})

winner_breakpoints_won = df.groupby('winner_id')[['w_break_points_won']].agg('count').reset_index()
winner_breakpoints_won = winner_breakpoints_won.rename(columns={'winner_id' : 'PlayerID'})



data_frames = [loser_breakpoints_won, winner_breakpoints_won]

breakpoints = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
breakpoints['breakpoint_count'] = breakpoints['l_break_points_won'] + breakpoints['w_break_points_won']
breakpoints

data_frames = [breakpoints, win_loss]
breakpoints = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
breakpoints['breakpoints_per_game'] = breakpoints['breakpoint_count'] / breakpoints['match_count']
columns_drop =['win_count', 'lose_count', 'match_count', 'win%']
breakpoints = breakpoints.drop(columns_drop, 1)
breakpoints


<ipython-input-14-07da347c4be1>:26: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  breakpoints = breakpoints.drop(columns_drop, 1)


,PlayerID,l_break_points_won,w_break_points_won,breakpoint_count,breakpoints_per_game
0,100644,71,149.0,220.0,1.000000
1,100754,0,0.0,0.0,0.000000
2,100943,0,0.0,0.0,0.000000
3,100997,0,NaN,NaN,NaN
4,101086,19,8.0,27.0,1.000000
...,...,...,...,...,...
2133,208597,0,NaN,NaN,NaN
2134,208937,0,NaN,NaN,NaN
2135,209226,0,NaN,NaN,NaN
2136,209238,0,NaN,NaN,NaN


In [15]:
### ACES
loss_aces = df.groupby('loser_id')[['l_ace']].agg('count').reset_index()
loss_aces = loss_aces.rename(columns={'loser_id' : 'PlayerID', 'lose_count': 'ace_count_lost'})

win_aces = df.groupby('winner_id')[['w_ace']].agg('count').reset_index()
win_aces = win_aces.rename(columns={'winner_id' : 'PlayerID', 'win_count':'ace_count_win'})

data_frames = [loss_aces, win_aces]

aces = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
aces['ace_count'] = aces['l_ace'] + aces['w_ace']

data_frames = [aces, win_loss]
aces = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
aces['aces_per_game'] = round((aces['ace_count'] / aces['match_count']),3)
columns_drop =['win_count', 'lose_count', 'match_count', 'win%']
aces = aces.drop(columns_drop, 1)
aces

<ipython-input-15-ce9849fa152a>:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  aces = aces.drop(columns_drop, 1)


,PlayerID,l_ace,w_ace,ace_count,aces_per_game
0,100644,71,149.0,220.0,1.00
1,100754,0,0.0,0.0,0.00
2,100943,0,0.0,0.0,0.00
3,100997,0,NaN,NaN,NaN
4,101086,19,8.0,27.0,1.00
...,...,...,...,...,...
2133,208597,0,NaN,NaN,NaN
2134,208937,0,NaN,NaN,NaN
2135,209226,0,NaN,NaN,NaN
2136,209238,0,NaN,NaN,NaN


There are only five players in the Top 50 of the ATP Rankings that hit on average at least one ace per service game in 2018. Aces are the ultimate serve weapon, but performance varies greatly at the pinnacle of our sport with the number of aces hit per game

In [16]:
## COMBINING ALL DATAFRAMES TOGETHER

from functools import reduce

data_frames = [PlayerData, win_loss, aces, serve_pts, breakpoints]

PD_merge = reduce(lambda  left,right: pd.merge(left,right,on=['PlayerID'],
                                            how='left'), data_frames)
PD_merge

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,win_count,lose_count,match_count,win%,l_ace,w_ace,ace_count,aces_per_game,l_first_serve,w_first_serve,l_first_serve_win,w_first_serve_win,l_break_points_won,w_break_points_won,breakpoint_count,breakpoints_per_game
0,103720,Lleyton Hewitt,20.752909,180.0,1.0,R,AUS,458.0,196.0,654.0,0.70,183.0,422.0,605.0,0.925,0.525558,0.539098,0.675974,0.773470,183.0,422.0,605.0,0.925076
1,102158,Patrick Rafter,26.477755,185.0,2.0,R,AUS,97.0,44.0,141.0,0.69,40.0,86.0,126.0,0.894,0.637279,0.653328,0.700358,0.795185,40.0,86.0,126.0,0.893617
2,103017,Nicolas Kiefer,22.535250,183.0,4.0,R,GER,217.0,162.0,379.0,0.57,150.0,205.0,355.0,0.937,0.514047,0.537990,0.689337,0.792370,150.0,205.0,355.0,0.936675
3,210013,Martin Damm Sr,28.227242,188.0,67.0,R,CZE,19.0,32.0,51.0,0.37,31.0,19.0,50.0,0.980,0.546553,0.611743,0.693080,0.806999,31.0,19.0,50.0,0.980392
4,103264,Mariano Puerta,26.904860,180.0,9.0,L,ARG,76.0,79.0,155.0,0.49,77.0,70.0,147.0,0.948,0.637432,0.680268,0.613399,0.746508,77.0,70.0,147.0,0.948387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2230,144984,Artem Dubrivnyy,20.700890,NaN,424.0,R,RUS,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.632353,NaN,0.465116,NaN,1.0,NaN,NaN,NaN
2231,200553,Alen Avidzba,19.635866,NaN,498.0,R,RUS,1.0,1.0,2.0,0.50,1.0,1.0,2.0,1.000,0.492537,0.625000,0.606061,0.777778,1.0,1.0,2.0,1.000000
2232,208230,Alibek Kachmazov,17.158111,NaN,1491.0,R,RUS,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.518072,NaN,0.534884,NaN,1.0,NaN,NaN,NaN
2233,127339,Borna Gojo,21.722108,NaN,280.0,R,CRO,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,0.616055,NaN,0.643789,NaN,2.0,NaN,NaN,NaN


In [17]:
columns_to_rename = {'l_ace' : 'aces_in_match_lost',
                    'w_ace': 'aces_in_match_won',
                    'l_first_serve': 'first_serve_percentage_match_lost',
                    'w_first_serve': 'first_serve_percentage_match_won',
                    'l_first_serve_win': 'first_serve_won_percentage_match_lost',
                    'w_first_serve_win': 'first_serve_won_percentage_match_won',
                    'l_break_points_won': 'breakpoints_won_match_lost',
                    'w_break_points_won': 'breakpoints_won_match_won'}
PD_merge = PD_merge.rename(columns = columns_to_rename)


In [22]:
### filtering for players who only play 50 matches and their max rank was over 100

PD_merge = PD_merge[PD_merge.match_count > 50]
PD_merge = PD_merge[PD_merge.MaxRank < 100]
PD_merge

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,win_count,lose_count,match_count,win%,aces_in_match_lost,aces_in_match_won,ace_count,aces_per_game,first_serve_percentage_match_lost,first_serve_percentage_match_won,first_serve_won_percentage_match_lost,first_serve_won_percentage_match_won,breakpoints_won_match_lost,breakpoints_won_match_won,breakpoint_count,breakpoints_per_game
0,103720,Lleyton Hewitt,20.752909,180.0,1.0,R,AUS,458.0,196.0,654.0,0.70,183.0,422.0,605.0,0.925,0.525558,0.539098,0.675974,0.773470,183.0,422.0,605.0,0.925076
1,102158,Patrick Rafter,26.477755,185.0,2.0,R,AUS,97.0,44.0,141.0,0.69,40.0,86.0,126.0,0.894,0.637279,0.653328,0.700358,0.795185,40.0,86.0,126.0,0.893617
2,103017,Nicolas Kiefer,22.535250,183.0,4.0,R,GER,217.0,162.0,379.0,0.57,150.0,205.0,355.0,0.937,0.514047,0.537990,0.689337,0.792370,150.0,205.0,355.0,0.936675
3,210013,Martin Damm Sr,28.227242,188.0,67.0,R,CZE,19.0,32.0,51.0,0.37,31.0,19.0,50.0,0.980,0.546553,0.611743,0.693080,0.806999,31.0,19.0,50.0,0.980392
4,103264,Mariano Puerta,26.904860,180.0,9.0,L,ARG,76.0,79.0,155.0,0.49,77.0,70.0,147.0,0.948,0.637432,0.680268,0.613399,0.746508,77.0,70.0,147.0,0.948387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,200282,Alex De Minaur,20.711841,183.0,18.0,R,AUS,45.0,27.0,72.0,0.62,26.0,45.0,71.0,0.986,0.615800,0.628429,0.638539,0.775507,26.0,45.0,71.0,0.986111
2065,126774,Stefanos Tsitsipas,20.788501,193.0,6.0,R,GRE,62.0,37.0,99.0,0.63,37.0,62.0,99.0,1.000,0.584801,0.624753,0.735252,0.784889,37.0,62.0,99.0,1.000000
2078,105815,Tennys Sandgren,27.482546,188.0,41.0,R,USA,25.0,30.0,55.0,0.45,30.0,25.0,55.0,1.000,0.600247,0.633775,0.692401,0.799943,30.0,25.0,55.0,1.000000
2081,126610,Matteo Berrettini,23.581109,196.0,8.0,R,ITA,29.0,25.0,54.0,0.54,25.0,29.0,54.0,1.000,0.610410,0.628175,0.702783,0.822480,25.0,29.0,54.0,1.000000


In [23]:
PD_merge.to_csv('PlayerData.csv')

In [19]:
PD_merge[PD_merge.MaxRank == 1]

,PlayerID,Player_Name,Age,Height,MaxRank,Hand,Country,win_count,lose_count,match_count,win%,aces_in_match_lost,aces_in_match_won,ace_count,aces_per_game,first_serve_percentage_match_lost,first_serve_percentage_match_won,first_serve_won_percentage_match_lost,first_serve_won_percentage_match_won,breakpoints_won_match_lost,breakpoints_won_match_won,breakpoint_count,breakpoints_per_game
0,103720,Lleyton Hewitt,20.752909,180.0,1.0,R,AUS,458.0,196.0,654.0,0.70,183.0,422.0,605.0,0.925,0.525558,0.539098,0.675974,0.773470,183.0,422.0,605.0,0.925076
8,102845,Carlos Moya,22.546201,190.0,1.0,R,ESP,297.0,167.0,464.0,0.64,162.0,282.0,444.0,0.957,0.576697,0.587575,0.674769,0.769395,162.0,282.0,444.0,0.956897
16,102856,Gustavo Kuerten,24.347707,190.0,1.0,R,BRA,202.0,106.0,308.0,0.66,94.0,184.0,278.0,0.903,0.552564,0.556285,0.718379,0.796136,94.0,184.0,278.0,0.902597
40,103498,Marat Safin,20.835044,193.0,1.0,R,RUS,319.0,180.0,499.0,0.64,161.0,292.0,453.0,0.908,0.552617,0.556719,0.707889,0.788261,161.0,292.0,453.0,0.907816
82,102338,Yevgeny Kafelnikov,25.221081,190.0,1.0,R,RUS,223.0,112.0,335.0,0.67,101.0,206.0,307.0,0.916,0.537804,0.560409,0.678835,0.773852,101.0,206.0,307.0,0.916418
85,101736,Andre Agassi,29.412731,180.0,1.0,R,USA,230.0,76.0,306.0,0.75,73.0,226.0,299.0,0.977,0.602926,0.624271,0.681651,0.770387,73.0,226.0,299.0,0.977124
133,103819,Roger Federer,22.485969,185.0,1.0,R,SUI,907.0,212.0,1119.0,0.81,202.0,866.0,1068.0,0.954,0.597457,0.629201,0.701538,0.796579,202.0,866.0,1068.0,0.954424
205,101948,Pete Sampras,27.550992,185.0,1.0,R,USA,119.0,49.0,168.0,0.71,47.0,116.0,163.0,0.970,0.588892,0.606749,0.746246,0.829313,47.0,116.0,163.0,0.970238
230,103507,Juan Carlos Ferrero,23.589322,183.0,1.0,R,ESP,353.0,190.0,543.0,0.65,183.0,332.0,515.0,0.948,0.637765,0.640235,0.629894,0.733483,183.0,332.0,515.0,0.948435
756,104745,Rafael Nadal,22.228611,185.0,1.0,L,ESP,741.0,146.0,887.0,0.84,142.0,724.0,866.0,0.976,0.680257,0.683340,0.637243,0.748767,142.0,724.0,866.0,0.976325


Feder, Murray, Nadal, Roddick, Djokovic

#win% likely wrong
aces_in_match_lost
match_wins & match_loss

l_first_serve = first serve percentage matches lost

ace_count / match_count() - check

In [20]:
# ### BULK FUNCTION TO BUILD OUT PLAYER DATABASE ### -

# PlayerColumns = ['PlayerID', 'Player_Name', 'Age', 'Height', 'MaxRank', 'Hand', 'Country', 'Matches', 'Aces', 'Double_Faults', 'First_Serve_%', 'First_Serve_Win_%', 'Breakpoints_Won']

# PlayerData = pd.DataFrame(columns = PlayerColumns)
# PlayerData.set_index('PlayerID')

# #Creating data for the PlayerData dataframe
# for index, row in df.iterrows():
#     #player was id as player_1
#     p1_id = int(row.player_1_id)
    
#     #checking
#     if (p1_id not in PlayerData.index):
#         PlayerData.loc[p1_id] = [p1_id, row.player_1_name, row.player_1_age, row.player_1_ht, row.player_1_rank, row.player_1_hand, row.player_1_ioc, row.player_1_counter, row.player_1_aces, row.player_1_double_faults, row.player_1_first_serve_percent, row.player_1_first_serve_win_percent, row.player_1_break_points_won]

#     #Updating best rank
#     #Update rank with best rank value
#     if(row.player_1_rank < PlayerData.loc[p1_id].MaxRank and not np.isnan(PlayerData.loc[p1_id].MaxRank)):
#         PlayerData.loc[p1_id] = [p1_id, row.player_1_name, row.player_1_age, row.player_1_ht, row.player_1_rank, row.player_1_hand, row.player_1_ioc, row.player_1_counter ,row.player_1_aces, row.player_1_double_faults, row.player_1_first_serve_percent, row.player_1_first_serve_win_percent, row.player_1_break_points_won]
        
#     else:
#         PlayerData.loc[p1_id] = [p1_id, row.player_1_name, row.player_1_age, row.player_1_ht, row.player_1_rank, row.player_1_hand, row.player_1_ioc, row.player_1_counter, row.player_1_aces, row.player_1_double_faults, row.player_1_first_serve_percent, row.player_1_first_serve_win_percent, row.player_1_break_points_won]
        
        
#     #same code but for player2 (as they were set randomly)
#     p2_id = int(row.player_2_id)
    
#     #checking
#     if (p2_id not in PlayerData.index):
#         PlayerData.loc[p2_id] = [p2_id, row.player_2_name, row.player_2_age, row.player_2_ht, row.player_2_rank, row.player_2_hand, row.player_2_ioc, row.player_2_counter, row.player_2_aces, row.player_2_double_faults, row.player_2_first_serve_percent, row.player_2_first_serve_win_percent, row.player_2_break_points_won]

#     #Updating best rank
#     #Update rank with best rank value
#     if(row.player_2_rank < PlayerData.loc[p2_id].MaxRank and not np.isnan(PlayerData.loc[p2_id].MaxRank)):
#         PlayerData.loc[p2_id] = [p2_id, row.player_2_name, row.player_2_age, row.player_2_ht, row.player_2_rank, row.player_2_hand, row.player_2_ioc, row.player_2_counter, row.player_2_aces, row.player_2_double_faults, row.player_2_first_serve_percent, row.player_2_first_serve_win_percent, row.player_2_break_points_won]
        
#     else:
#         PlayerData.loc[p2_id] = [p2_id, row.player_2_name, row.player_2_age, row.player_2_ht, row.player_2_rank, row.player_2_hand, row.player_2_ioc, row.player_2_counter, row.player_2_aces, row.player_2_double_faults, row.player_2_first_serve_percent, row.player_2_first_serve_win_percent, row.player_2_break_points_won]

In [21]:
import re
def total_games(s):
    num_list = re.findall('\d+', s)
    num_list = [int(x) for x in num_list]
    return sum(num_list)

df['total_games'] = df['score'].apply(total_games)
df

,Unnamed: 0,tourney_id,tourney_name,surface,draw_size,tourney_level,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,w_first_serve,l_first_serve,w_first_serve_win,l_first_serve_win,w_break_points_won,l_break_points_won,total_games
tourney_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-01-11,108,1999-338,Sydney,Hard,32,A,16,103720,NaN,NaN,Lleyton Hewitt,R,180.0,AUS,17.878166,102158,2.0,NaN,Patrick Rafter,R,185.0,AUS,26.036961,7-6(1) 6-1,3,R32,84.0,2.0,0.0,73.0,51.0,32.0,14.0,10.0,4.0,6.0,0.0,5.0,59.0,36.0,24.0,8.0,9.0,3.0,7.0,104.0,456.0,4.0,3315.0,0.698630,0.610169,0.627451,0.666667,4.0,2.0,21
1999-01-11,107,1999-338,Sydney,Hard,32,A,15,103017,NaN,NaN,Nicolas Kiefer,R,183.0,GER,21.519507,210013,NaN,Q,Martin Damm Sr,R,188.0,CZE,26.444901,5-7 6-3 6-2,3,R32,115.0,6.0,4.0,86.0,38.0,31.0,32.0,15.0,2.0,3.0,8.0,2.0,84.0,52.0,40.0,11.0,14.0,3.0,6.0,36.0,1007.0,75.0,657.0,0.441860,0.619048,0.815789,0.769231,3.0,1.0,29
1999-01-11,106,1999-338,Sydney,Hard,32,A,14,103264,NaN,NaN,Mariano Puerta,L,180.0,ARG,20.312115,101733,NaN,NaN,Jan Siemerink,L,183.0,NED,28.744695,6-2 6-4,3,R32,61.0,1.0,1.0,46.0,31.0,25.0,11.0,9.0,0.0,0.0,1.0,7.0,51.0,24.0,15.0,14.0,9.0,3.0,6.0,38.0,983.0,19.0,1664.0,0.673913,0.470588,0.806452,0.625000,3.0,0.0,18
1999-01-11,105,1999-338,Sydney,Hard,32,A,13,101774,8.0,NaN,Todd Martin,R,198.0,USA,28.511978,102271,NaN,NaN,Hicham Arazi,L,175.0,MAR,25.229295,6-1 6-3,3,R32,43.0,10.0,0.0,36.0,27.0,25.0,7.0,8.0,0.0,0.0,2.0,3.0,47.0,27.0,16.0,6.0,8.0,1.0,5.0,16.0,1774.0,34.0,1069.0,0.750000,0.574468,0.925926,0.592593,4.0,0.0,16
1999-01-11,104,1999-338,Sydney,Hard,32,A,12,102845,3.0,NaN,Carlos Moya,R,190.0,ESP,22.373717,102563,NaN,NaN,Thomas Johansson,R,180.0,SWE,23.802875,6-3 7-6(4),3,R32,85.0,7.0,1.0,79.0,37.0,28.0,22.0,11.0,5.0,7.0,8.0,2.0,58.0,26.0,19.0,17.0,10.0,2.0,5.0,5.0,3159.0,17.0,1761.0,0.468354,0.448276,0.756757,0.730769,3.0,2.0,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-23,2655,2019-M-DC-2019-FLS-M-RUS-CAN-01,Davis Cup Finals SF: RUS vs CAN,Hard,4,D,1,126094,NaN,NaN,Andrey Rublev,R,188.0,RUS,22.078029,105577,NaN,NaN,Vasek Pospisil,R,193.0,CAN,29.404517,6-4 6-4,3,RR,85.0,3.0,3.0,67.0,44.0,37.0,9.0,10.0,5.0,6.0,16.0,3.0,65.0,44.0,29.0,7.0,10.0,4.0,7.0,23.0,1584.0,150.0,360.0,0.656716,0.676923,0.840909,0.659091,3.0,1.0,20
2019-11-23,2656,2019-M-DC-2019-FLS-M-RUS-CAN-01,Davis Cup Finals SF: RUS vs CAN,Hard,4,D,2,133430,NaN,NaN,Denis Shapovalov,L,185.0,CAN,20.594114,111575,NaN,NaN,Karen Khachanov,R,198.0,RUS,23.493498,6-4 4-6 6-4,3,RR,110.0,11.0,6.0,93.0,58.0,46.0,14.0,15.0,5.0,7.0,5.0,2.0,85.0,59.0,42.0,14.0,15.0,2.0,5.0,15.0,2050.0,17.0,1840.0,0.623656,0.694118,0.793103,0.711864,3.0,2.0,30
2019-11-23,2651,2019-M-DC-2019-FLS-M-GBR-ESP-01,Davis Cup Finals SF: GBR vs ESP,Hard,4,D,1,106378,NaN,NaN,Kyle Edmund,R,188.0,GBR,24.859685,103852,NaN,NaN,Feliciano Lopez,L,188.0,ESP,38.160164,6-3 7-6(3),3,RR,81.0,9.0,0.0,67.0,46.0,34.0,16.0,11.0,2.0,2.0,8.0,3.0,57.0,37.0,30.0,9.0,10.0,1.0,2.0,69.0,800.0,62.0,888.0,0.686567,0.649123,0.739130,0.810811,1.0,0.0,25
